In [4]:
## Version futuresales lib required = 0.1.8

# !pip install -i https://test.pypi.org/simple/ futuresales_denissimo==0.1.8

Looking in indexes: https://test.pypi.org/simple/


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm._tqdm_notebook import tqdm_notebook
from tqdm._tqdm_notebook import tqdm
tqdm_notebook.pandas()

import pickle

import futuresales as fs

In [6]:
from futuresales.utils import load_credentials
import neptune.new as neptune

cred = load_credentials('../credentials.json')['projects']['fs-feature-space']

run = neptune.init(
    api_token=cred['api_token'],
    project=cred['project'],
)  # your credentials


Info (NVML): Driver Not Loaded. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://app.neptune.ai/denissimo/fs-feature-space/e/FSFEAT-1
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [7]:
CONFIG = {
    'FETCH_DATA': False
}

In [8]:
run['dataset_dependecy'] = {
    'original': 'kaggle competitions download -c competitive-data-science-predict-future-sales',
}

In [9]:
df_provider = fs.distribution.DatasetProvider()

df_provider.file_list = [
    '/home/denissimo/Repo/fs_project/datasets/sample_submission.csv',
    '/home/denissimo/Repo/fs_project/datasets/test.csv',
    '/home/denissimo/Repo/fs_project/datasets/shops.csv',
    '/home/denissimo/Repo/fs_project/datasets/item_categories.csv',
    '/home/denissimo/Repo/fs_project/datasets/sales_train.csv',
    '/home/denissimo/Repo/fs_project/datasets/items.csv'
]

datasets = df_provider.get_dataset()

In [10]:
sales = datasets['sales_train.csv']
items = datasets['items.csv']
categories = datasets['item_categories.csv']
shops = datasets['shops.csv']

sales.date = sales.date.astype('datetime64[ns]')


from datetime import date
sales = sales.loc[sales.date < np.datetime64(date(2015, 11, 1))]

print("Before:", sales.shape)

sales_train = sales[
    (sales["item_cnt_day"] < 550)
    & (sales["item_price"] > 0)
    & (sales["item_price"] < 60000)
].copy()
print("After:", sales_train.shape)

sales.head()

Before: (2896782, 6)
After: (2896775, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.00,1.0
1,2013-03-01,0,25,2552,899.00,1.0
2,2013-05-01,0,25,2552,899.00,-1.0
3,2013-06-01,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [11]:
from extraction_configs.pipeline_configs import make_indexes, make_task_df
from extraction_configs.generators_config import make_baseline_train_generator
from extraction_configs.aggregation_configs import make_baseline_train_aggregation, make_baseline_submission_aggregation
from futuresales.distribution import to_pickle, from_pickle
from futuresales.features import FeatureExtractor

pipelines = {
    'idx': make_indexes(shops, categories),
}

task_df = {}
task_df['idx'] = pipelines['idx']['id_merging_stage'](items)

task_df['idx'].to_csv('../tmp/idx.csv')

pipelines['task_df'] = make_task_df(items, shops, categories, task_df['idx'])

In [12]:
from futuresales.pipeline import Pipeline

pipeline = Pipeline(
    tasks=pipelines['task_df'], 
    task_queue = [
        'id_merging_stage',
        'summarizing_and_name_merging_stage',
        'date_block_num_renaming',
        'object_id_encoding',
    ]
)

pipeline_test = pipeline(sales)
pipeline_train = pipeline(sales_train)

if not CONFIG['FETCH_DATA']:
    _ = pipeline_test.proceed_all()
    _ = pipeline_train.proceed_all()
    task_df['test'] = pipeline_test.result_storage['object_id_encoding']
    task_df['train'] = pipeline_train.result_storage['object_id_encoding']
    task_df['test'].to_csv('../tmp/task_df_test.csv')
    task_df['train'].to_csv('../tmp/task_df_train.csv')
else:
    task_df['test'] = pd.read_csv('../tmp/task_df_test.csv')
    task_df['train'] = pd.read_csv('../tmp/task_df_train.csv')

Stage - id_merging_stage complete
Stage - summarizing_and_name_merging_stage complete
Stage - date_block_num_renaming complete
Stage - object_id_encoding complete
Stage - id_merging_stage complete
Stage - summarizing_and_name_merging_stage complete
Stage - date_block_num_renaming complete
Stage - object_id_encoding complete


In [13]:
def dyn_agg(months):
    return {f'dynamic_aggregation_{k}_win_{months}':f'dynamic_aggregation_{k}_all' for k in ['mean', 'std', 'min', 'max']}

baseline = {}

file_map = {
    30: 'train_set',
    31: 'test_set',
    32: 'validation_set',
}

for train_max in range(30, 33):
    print(f'\nProcessing set - {file_map[train_max]}')
    baseline_extractor = FeatureExtractor(
        make_baseline_train_generator(train_max), 
        make_baseline_train_aggregation(train_max), 
        ['id'])
    baseline[file_map[train_max]] = baseline_extractor({
        'original': task_df['train'],
        'test': task_df['test']
        })
    features = baseline[file_map[train_max]]['features']
    for key in features:
        if not key in ['valid_target', 'target', 'lags']:
            features[key].columns = [
                f'{key}_{agg_type}' for agg_type in features[key].columns.values
                ]
    features = pd.concat(
        features.values(), axis=1, join='inner')
    features = features.rename(dyn_agg(train_max + 1), axis=1)
    features.to_csv(f'../tmp/baseline_{file_map[train_max]}.csv')


Processing set - train_set
Subseries stage: id_sales
Subseries stage: id_sales_test
Subseries stage: train_series
Subseries stage: diff_1
Subseries stage: diff_2
Subseries stage: lags
Subseries stage: lags_12

Processing set - test_set
Subseries stage: id_sales
Subseries stage: id_sales_test
Subseries stage: train_series
Subseries stage: diff_1
Subseries stage: diff_2
Subseries stage: lags
Subseries stage: lags_12

Processing set - validation_set
Subseries stage: id_sales
Subseries stage: id_sales_test
Subseries stage: train_series
Subseries stage: diff_1
Subseries stage: diff_2
Subseries stage: lags
Subseries stage: lags_12


In [14]:
feature_info = {
    key: baseline[file_map[30]]['features'][key].columns.to_list() for key in baseline[file_map[30]]['features'].keys()
}

In [15]:
run['features'] = feature_info
run['extraction_configs'].upload_files([
    '../featuring/extraction_configs/aggregation_configs.py',
    '../featuring/extraction_configs/pipeline_configs.py',
    '../featuring/extraction_configs/generators_config.py',
    ])

In [17]:
run['extraction_notebook'].upload_files('../featuring/baseline_extraction.ipynb')

In [20]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.


All 1 operations synced, thanks for waiting!
